In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import datetime

In [2]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [3]:
def load_data():
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    return train_df, test_df

In [4]:
train_df, test_df = load_data()

In [5]:
train_df.head()

,price,desc,ad_id,location,timestamp,mileage,brand,model,year,fuel,transmission,color,car_type
0,569000,"""เลขไมล์5,700กิโลแท้ 1.3 SP โฉมปัจจุบัน รถ2เดื...",354186401,กรุงเทพมหานคร,1584521455,5700,Mazda,Mazda2 Sport,2020,เบนซิน,เกียร์อัตโนมัติ,เทา,รถเก๋ง
1,1150000,ขาย Chevrolet c10 ปี 1965 รถแท้ ขับขวา พร้อมโอ...,356227846,กรุงเทพมหานคร,1591021387,1234,Chevrolet,รุ่นอื่นๆ,1965,เบนซิน,เกียร์อัตโนมัติ,แดง,รถอื่น ๆ
2,799000,📢มาแว้วว... ลค.ที่ชอบรถสีขาว​ ใสๆเลย\nใครเร็วก...,356055202,นนทบุรี,1590502642,100000,Mitsubishi,Pajero Sport,2016,ดีเซล,เกียร์อัตโนมัติ,ขาว,รถ MPV / SUV
3,429000,รหัส 130/v36\n2010 toyota CAMRY 2.4 hybrid nav...,356169430,กรุงเทพมหานคร,1590844871,184921,Toyota,Camry,2010,ไฮบริด,เกียร์อัตโนมัติ,ดำ,รถเก๋ง
4,268000,เครื่อง 1.2 E โฉมไฟใหม่ พ.พาวเวอร์ ก.ไฟฟ้า ซทล...,354415016,กรุงเทพมหานคร,1585546785,103000,Nissan,Almera,2016,เบนซิน,เกียร์อัตโนมัติ,ขาว,รถเก๋ง


In [6]:
test_df.head()

,price,desc,ad_id,location,timestamp,mileage,brand,model,year,fuel,transmission,color,car_type
0,549000,"Option: เครื่องเล่น CD, USB, กระจกไฟฟ้า, เซ็นท...",354696215,นนทบุรี,1586421506,31200,Ford,Ranger,2017,ดีเซล,เกียร์อัตโนมัติ,ดำ,รถกระบะ
1,358000,ขายรถบ้าน 2006 ISUZU MU-7 แรงๆๆ (เงินสดลดได้)\...,354237756,สมุทรปราการ,1584748598,267000,Isuzu,MU-7,2006,ดีเซล,เกียร์อัตโนมัติ,บรอนซ์เงิน,รถ MPV / SUV
2,529000,🔥แคปเปิดได้ 🔥รุ่นใหม่ล่าสุด 🔥แรง ประหยัด เครื่...,352545121,กรุงเทพมหานคร,1577353632,8,Toyota,Hilux Revo,2019,ดีเซล,เกียร์ธรรมดา,ขาว,รถกระบะ
3,1135000,TOYOTA NEW FORTUNER 2.8 V 4WD SUV AUTO ปี 2015...,356194406,เชียงใหม่,1590918420,100000,Toyota,Fortuner,2015,ดีเซล,เกียร์อัตโนมัติ,ขาว,รถ MPV / SUV
4,469000,💢 ออกรถ 999 บ. 💢\n#ฟรี#ออกรถพร้อมประกันตัวรถ 1...,355315874,นนทบุรี,1588235606,NaN,Toyota,Hilux Vigo,2015,ดีเซล,เกียร์ธรรมดา,เทา,รถกระบะ


In [7]:
train_df.isna().sum()

price              0
desc             171
ad_id              0
location           0
timestamp          0
mileage         1662
brand              0
model              0
year               0
fuel               0
transmission     213
color            255
car_type           0
dtype: int64

In [8]:
test_df.isna().sum()

price             0
desc             43
ad_id             0
location          0
timestamp         0
mileage         416
brand             0
model             0
year              0
fuel              0
transmission     54
color            69
car_type          0
dtype: int64

# Feature Engineering

In [9]:
def fix_mileage(df):
    fix = df.copy()
    def fix_(x):
        try:
            x = int(x)
        except:
            x = np.nan
        return x
    fix['mileage'] = fix['mileage'].apply(lambda x: fix_(x))
    return fix

In [10]:
train_df_fix = fix_mileage(train_df)
test_df_fix = fix_mileage(test_df)

In [11]:
def fill_na(df):
    fill_df = df.copy()
    transmission_mode = fill_df['transmission'].mode()
    color_mode = fill_df['color'].mode()
    fill_df['desc'] = fill_df['desc'].fillna('')     
    fill_df['transmission'] = fill_df['transmission'].fillna(transmission_mode)
    fill_df['color'] = fill_df['color'].fillna(color_mode)
    #MILEAGE
#     nan_rows = fill_df['mileage'].isna()
#     np.random.seed(0)
#     random_age = np.random.choice(fill_df['mileage'][~nan_rows], replace=True, size=sum(nan_rows))
#     fill_df.loc[nan_rows,'mileage'] = random_age
    fill_df['mileage'] = fill_df['mileage'].fillna(fill_df['mileage'].mode()[0])
    return fill_df

In [12]:
train_df_filled = fill_na(train_df_fix)
test_df_filled = fill_na(test_df_fix)

In [13]:
def extract_data(df):
    feat = df.copy()    
    #DESC
#     feat['len_desc'] = feat['desc'].apply(lambda x: len(str(x)))
    
    #MILEAGE
    q1 = feat['mileage'].quantile(0.25)
    q3 = feat['mileage'].quantile(0.75)
    IQR = q3 - q1
    upper_limit = q3 + 1.5*IQR
    lower_limit = q1 - 1.5*IQR
    maybe_outlier = 10**7
    outlier_rows = (feat['mileage'] > upper_limit) | (feat['mileage'] < lower_limit)  
    feat.loc[outlier_rows, 'mileage'] = feat['mileage'][~outlier_rows].mean()
#     nan_rows = feat['mileage'].isna()  
#     np.random.seed(0)
#     random_age = np.random.choice(feat['mileage'][~nan_rows], replace=True, size=sum(nan_rows))
#     feat.loc[nan_rows,'mileage'] = random_age    

    #TIMESTAMP
    feat['timestamp'] = feat['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x))
#     feat['time_year'] = feat['timestamp'].apply(lambda x: x.year)
    
    #AGE
#     feat['car_age'] = (feat['time_year'] - feat['year'])
    
    #LOCATION
    def sort_location(x):
        if x not in ['กรุงเทพมหานคร']:
            x = 'Other City'
        return x
    feat['location'] = feat['location'].apply(lambda x: sort_location(x))
    
    #YEAR
    def sort_year(x):
        if x<1990:
            x='before 1990'
        return str(x)
    feat['year'] = feat['year'].apply(lambda x: sort_year(x))
    
    #BRAND
#     feat['brand_class'] = feat['brand'].apply(lambda x: 'high_class' if x in ['Porsche', 'ยี่ห้ออื่นๆ', 'Audi', 'Land Rover', 'BMW', 'Mercedes-Benz'] else 'medium_class' )
    return feat

In [14]:
train_data = extract_data(train_df_filled)
test_data = extract_data(test_df_filled)

In [15]:
def drop_columns(df, drop_col):
    drop_feat = df.copy()
    drop_feat = drop_feat.drop(drop_col, axis=1)
    return drop_feat

In [16]:
drop_col = ['desc','ad_id','timestamp','model','color','year']
train_data_dropped = drop_columns(train_data, drop_col)
test_data_dropped = drop_columns(test_data, drop_col)

In [17]:
train_data_dropped

,price,location,mileage,brand,fuel,transmission,car_type
0,569000,กรุงเทพมหานคร,5700.000000,Mazda,เบนซิน,เกียร์อัตโนมัติ,รถเก๋ง
1,1150000,กรุงเทพมหานคร,1234.000000,Chevrolet,เบนซิน,เกียร์อัตโนมัติ,รถอื่น ๆ
2,799000,Other City,100000.000000,Mitsubishi,ดีเซล,เกียร์อัตโนมัติ,รถ MPV / SUV
3,429000,กรุงเทพมหานคร,184921.000000,Toyota,ไฮบริด,เกียร์อัตโนมัติ,รถเก๋ง
4,268000,กรุงเทพมหานคร,103000.000000,Nissan,เบนซิน,เกียร์อัตโนมัติ,รถเก๋ง
...,...,...,...,...,...,...,...
7098,1990000,กรุงเทพมหานคร,100000.000000,Mercedes-Benz,เบนซิน,เกียร์อัตโนมัติ,รถเก๋ง
7099,299000,กรุงเทพมหานคร,169376.000000,Mitsubishi,ดีเซล,เกียร์ธรรมดา,รถกระบะ
7100,1290000,กรุงเทพมหานคร,89000.000000,BMW,ดีเซล,เกียร์อัตโนมัติ,รถ MPV / SUV
7101,479000,Other City,100000.000000,Honda,เบนซิน,เกียร์อัตโนมัติ,รถ MPV / SUV


In [18]:
def extract_feature(df):
    feat = df.copy()
    numeric_feature = ['mileage', 'length_desc']
    catogorical_feature = ['brand','transmission','location','car_type']
    mlb = MultiLabelBinarizer() 
    fuel_col = pd.DataFrame(mlb.fit_transform(s.split(' + ') for s in feat['fuel']), columns=['fuel_1','fuel_2','fuel_3','fuel_4','fuel_5'])
    feat = pd.concat([feat, fuel_col], axis=1).drop('fuel', axis=1)
    feat = pd.get_dummies(feat, columns=catogorical_feature)
    return feat 

In [19]:
train_feature = extract_feature(train_data_dropped)
test_feature = extract_feature(test_data_dropped)

In [20]:
train_feature.head()

,price,mileage,fuel_1,fuel_2,fuel_3,fuel_4,fuel_5,brand_Audi,brand_BMW,brand_Chevrolet,...,brand_ยี่ห้ออื่นๆ,transmission_เกียร์ธรรมดา,transmission_เกียร์อัตโนมัติ,location_Other City,location_กรุงเทพมหานคร,car_type_รถ MPV / SUV,car_type_รถกระบะ,car_type_รถตู้,car_type_รถอื่น ๆ,car_type_รถเก๋ง
0,569000,5700.0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
1,1150000,1234.0,0,0,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
2,799000,100000.0,0,0,1,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
3,429000,184921.0,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1
4,268000,103000.0,0,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [21]:
test_feature.head()

,price,mileage,fuel_1,fuel_2,fuel_3,fuel_4,fuel_5,brand_Audi,brand_BMW,brand_Chevrolet,...,brand_ยี่ห้ออื่นๆ,transmission_เกียร์ธรรมดา,transmission_เกียร์อัตโนมัติ,location_Other City,location_กรุงเทพมหานคร,car_type_รถ MPV / SUV,car_type_รถกระบะ,car_type_รถตู้,car_type_รถอื่น ๆ,car_type_รถเก๋ง
0,549000,31200.000000,0,0,1,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
1,358000,100918.017503,0,0,1,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
2,529000,8.000000,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
3,1135000,100000.000000,0,0,1,0,0,0,0,0,...,0,0,1,1,0,1,0,0,0,0
4,469000,100000.000000,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,1,0,0,0


# Train Model

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression

In [23]:
X = train_feature.drop('price', axis=1)
y = train_feature['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
# y_train = np.sqrt(y_train)
# y_val = np.sqrt(y_val)
# y = np.sqrt(y)

In [25]:
X_test = test_feature.drop('price', axis=1)
y_test = test_feature['price']

In [26]:
# y_test = np.sqrt(y_test)

In [27]:
def train_model(X_train, y_train):
    model =GradientBoostingRegressor(random_state=0, learning_rate=0.1, n_estimators=100, max_depth=7)
    model.fit(X_train, y_train)
    return model

In [28]:
model = train_model(X_train, y_train)

In [29]:
feat_imp = pd.DataFrame([X_train.columns, model.feature_importances_]).T
feat_imp.columns = ['feature','importance']

In [30]:
feat_imp.sort_values(by='importance', ascending=False).head(10)

,feature,importance
23,brand_Porsche,0.288302
0,mileage,0.259698
19,brand_Mercedes-Benz,0.0684995
7,brand_BMW,0.0630649
38,car_type_รถอื่น ๆ,0.0523396
3,fuel_3,0.0373601
34,location_กรุงเทพมหานคร,0.0328408
32,transmission_เกียร์อัตโนมัติ,0.0313923
33,location_Other City,0.0265894
4,fuel_4,0.0250976


In [31]:
feat_imp.sort_values(by='importance', ascending=False).tail(10)

,feature,importance
11,brand_Hyundai,0.000263057
18,brand_Mazda,0.000211409
10,brand_Honda,0.000193594
17,brand_MG,0.000192058
28,brand_Volkswagen,0.000152401
12,brand_Isuzu,0.000116487
2,fuel_2,0.000113665
24,brand_Proton,0.000109684
14,brand_Kia,2.02959e-05
13,brand_Jeep,0


In [32]:
prediction = model.predict(X_val)

In [33]:
def eval_acc(y_val, prediction):
    acc = mean_squared_error(y_val, prediction)
    acc2 = r2_score(y_val, prediction)
    print('RMSE :', np.sqrt(acc))
    print('R-Squared: ', acc2)
    return (acc, acc2)

In [34]:
def pipeline():
    model = train_model(X_train, y_train)
    prediction = model.predict(X_val)
    acc = eval_acc(y_val, prediction)
    return acc

In [35]:
acc = pipeline()

RMSE : 777016.8500447132
R-Squared:  -0.5818551306678426


In [36]:
def pipeline2():
    model = train_model(X, y)
    prediction = model.predict(X_test)
    acc = eval_acc(y_test, prediction)
    return acc

In [37]:
acc2 = pipeline2()

RMSE : 407144.7592868134
R-Squared:  0.5541116978326366


# Recursive feature elimination with cross-validation

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
est= GradientBoostingRegressor(random_state=0, learning_rate=0.1, n_estimators=100, max_depth=7)

rfecv = RFECV(estimator=est, step=1, cv=5,scoring='neg_mean_squared_error')
rfecv.fit(X_train, y_train)

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

In [ ]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
df2 = pd.DataFrame()
df2['featname'] = X_train.columns
df2['rank'] = rfecv.ranking_
df2.sort_values(by='rank').head(40)['featname'].values